In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import metrics,preprocessing
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,RandomForestRegressor
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


In [ ]:
df =  pd.read_pickle('../data.2021-03-01.scenarioJunction.pkl.gz').drop(['meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed','meta_id'], axis=1, errors="ignore")

In [ ]:
df.dropna(axis=1, how='all',inplace=True)

In [ ]:
df_red = df[df['action_team'] == 'red'].copy().drop('action_team', axis=1)
df_blue = df[df['action_team'] == 'blue'].copy().drop('action_team', axis=1)
df_red['label'] = df_red['winner'].apply(lambda x: 1 if x == 'red' else -1)
df_blue['label'] = df_blue['winner'].apply(lambda x: 1 if x == 'blue' else -1)

df_new=pd.concat([df_red,df_blue])

df_neg=df_new[df_new['label']==-1]
df_pos=df_new[df_new['label']==1]

df_move=df_new.loc[((df_new['action_type_Move'] == True)|(df_new['action_type_MoveLoadInto'] == True))].copy()
df_attack=df_new.loc[((df_new['action_type_Attack'] == True)| (df_new['action_type_AttackGround'] == True) |(df_new['action_type_AttackRespond'] == True))].copy()
df_pass=df_new.loc[((df_new['action_type_Pass'] == True)| (df_new['action_type_PassFigure'] == True) |(df_new['action_type_PassTeam'] == True) | (df_new['action_type_PassRespond'] == True))].copy()

df_red_move=df_red.loc[((df_red['action_type_Move'] == True)|(df_red['action_type_MoveLoadInto'] == True))].copy()
df_red_attack=df_red.loc[((df_red['action_type_Attack'] == True)| (df_red['action_type_AttackGround'] == True) |(df_red['action_type_AttackRespond'] == True))].copy()
df_red_pass=df_red.loc[((df_red['action_type_Pass'] == True)| (df_red['action_type_PassFigure'] == True) |(df_red['action_type_PassTeam'] == True) | (df_red['action_type_PassRespond'] == True))].copy()

df_blue_move=df_blue.loc[((df_blue['action_type_Move'] == True)|(df_blue['action_type_MoveLoadInto'] == True))].copy()
df_blue_attack=df_blue.loc[((df_blue['action_type_Attack'] == True)| (df_blue['action_type_AttackGround'] == True) |(df_blue['action_type_AttackRespond'] == True))].copy()
df_blue_pass=df_blue.loc[((df_blue['action_type_Pass'] == True)| (df_blue['action_type_PassFigure'] == True) |(df_blue['action_type_PassTeam'] == True) | (df_blue['action_type_PassRespond'] == True))].copy()



def CrossValidatedPredictions(df):

    lr = linear_model.LinearRegression()
    X = df.drop(['winner', 'label'], axis=1)
    y = df['label']

    # cross_val_predict returns an array of the same size as `y` where each entry
    # is a prediction obtained by cross validation:
    predicted = cross_val_predict(lr, X, y, cv=10)

    fig, ax = plt.subplots()
    ax.scatter(y, predicted)
    #ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.show()


In [ ]:
def crossValidatedPredictionsRegressor(df):

    regr = RandomForestRegressor()
    X = df.drop(['winner', 'label'], axis=1)
    y = df['label']

    # cross_val_predict returns an array of the same size as `y` where each entry
    # is a prediction obtained by cross validation:
    predicted = cross_val_predict(regr, X, y, cv=10)

    fig, ax = plt.subplots()
    ax.scatter(y, predicted,s=0.1)
    #ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.show()


In [ ]:
crossValidatedPredictionsRegressor(df_new)

In [ ]:
crossValidatedPredictionsRegressor(df_neg)

In [ ]:
crossValidatedPredictionsRegressor(df_pos)

In [ ]:
crossValidatedPredictionsRegressor(df_red)

In [ ]:
crossValidatedPredictionsRegressor(df_blue)

In [ ]:
crossValidatedPredictionsRegressor(df_move)

In [ ]:
crossValidatedPredictionsRegressor(df_attack)

In [ ]:
crossValidatedPredictionsRegressor(df_pass)

In [ ]:
crossValidatedPredictionsRegressor(df_red_move)

In [ ]:
crossValidatedPredictionsRegressor(df_red_attack)

In [ ]:
crossValidatedPredictionsRegressor(df_red_pass)

In [ ]:
crossValidatedPredictionsRegressor(df_blue_move)

In [ ]:
crossValidatedPredictionsRegressor(df_blue_attack)

In [ ]:
crossValidatedPredictionsRegressor(df_blue_pass)

In [ ]:
def random_regressor(df):
    X = df.drop(['winner', 'label'], axis=1)
    y = df['label']
    train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.25, random_state = 42)
    regr = RandomForestRegressor()
    regr.fit(X, y)
    predictions = regr.predict(test_features)

    # Calculate the absolute errors
    errors = abs(predictions - test_labels)
    # Print out the mean absolute error (mae)
    print('Mean Absolute Error:', round(np.mean(errors), 2))
    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errors / test_labels)
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print('Mean Absolute Error:', metrics.mean_absolute_error(test_labels, predictions))
    print('Mean Squared Error:', metrics.mean_squared_error(test_labels, predictions))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_labels, predictions)))


In [ ]:
random_regressor(df_new)

In [ ]:
random_regressor(df_red)

In [ ]:
random_regressor(df_blue)

In [ ]:
random_regressor(df_neg)

In [ ]:
random_regressor(df_pos)

In [ ]:
random_regressor(df_move)

In [ ]:
random_regressor(df_attack)

In [ ]:
random_regressor(df_pass)

In [ ]:
random_regressor(df_red_move)

In [ ]:
random_regressor(df_red_attack)

In [ ]:
random_regressor(df_pass)

In [ ]:
random_regressor(df_blue_move)

In [ ]:
random_regressor(df_blue_attack)

In [ ]:
random_regressor(df_blue_pass)

In [ ]:
def random_forest(df):
    f_select=[]
   
    X = df.drop(['winner', 'label'], axis=1)
    y = df['label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    clf = RandomForestClassifier(n_estimators=100)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    proba = clf.predict_proba(X_test)
    proba = np.around(proba, decimals=2)
    print(y_train)  


    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

    # feature importanti per determinarlo
    feature_imp = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
    print('Feature importance:\n', feature_imp)
    f_selection=feature_imp[(feature_imp>0)]
    print('Feature selection:\n',f_selection)
            
        
    # Creating a bar plot
    sns.barplot(x=f_selection, y=f_selection.index)
    # Add labels to your graph
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("Visualizing Important Features")
    plt.legend()
    plt.show()
    #return X_test, y_test, y_pred, proba


In [ ]:
random_forest(df_new)